In [ ]:
import warnings
warnings.filterwarnings('ignore')
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Gate
import numpy as np
#import other files
import import_ipynb
import Composite_Gates_final
import Loop_Operations_final
import Measurements
import Elementary_Gates

In [ ]:
# Quantum Counting - determine Inputs

In [ ]:
# prepare circuit as it would be created using IBM Composer / QASM

def qft(size, control_qubits=None, inverse=True):
        #size=len(target_qubits)
        n = size
        circuit = QuantumCircuit(n)
        def swap_registers(circuit, n):
            for qubit in range(n//2):
                circuit.swap(qubit, n-qubit-1)
            return circuit
        def qft_rotations(circuit, n):
            if n == 0:
                return circuit
            n -= 1
            circuit.h(n)
            for qubit in range(n):
                circuit.cp(np.pi/2**(n-qubit), qubit, n)
            qft_rotations(circuit, n)

        qft_rotations(circuit, n)
        swap_registers(circuit, n)
        qft_gate = circuit.to_gate()
        qft_gate.label="QFT"
        if inverse==True:
            qft_gate=qft_gate.inverse()
            qft_gate.label="QFT_inv"
        if control_qubits!=None:
            for i in range(len(control_qubits)):
                qft_gate=qft_gate.control()
        return qft_gate
    
    
qubits = 8

qr1=QuantumRegister(int(qubits/2), name="q0")
qr2=QuantumRegister(int(qubits/2), name="q1")
qc=QuantumCircuit(qr1,qr2, ClassicalRegister(int(qubits/2))) #means qubits: [0-7], cbits: [0-3]

#create objects for quantum operations
e_gate=Elementary_Gates.ElementaryGate()
c_gate=Composite_Gates_final.CompositeGate()
l_gate=Loop_Operations_final.LoopOperation()
m_gate=Measurements.MeasurementGate()

#initial Hadamards
target_qubits=list(range(qubits))
qc.h(target_qubits)

#Grover unitaries
c_grover = c_gate.grover(int(qubits/2), control_qubits=[0])
for i in range(0,int(qubits/2)):
    for j in range(2**i):
        qc.append(c_grover, [i] + list(range(int(qubits/2), qubits)))

#QFT-inv
qc_qft = QuantumCircuit(int(qubits/2))
qc_qft.append(qft(int(qubits/2)), list(range(int(qubits/2))))
qc1 = qc.compose(qc_qft.decompose(), list(range(int(qubits/2))))

qc1.measure(range(int(qubits/2)), range(int(qubits/2)))

In [ ]:
# calculate inputs for metrices

def all_actions(qc):
    actions = []
    for gate in qc.data:
        indices = [(gate[1][i].index, gate[1][i].register.name) for i in range(len(gate[1]))]
        for index in indices:
            actions.append(index)
    return actions

def count_params(qubits):
    un_params = qubits -1
    ov_params = 0
    for i in range(qubits):
        ov_params+=i
    return un_params, ov_params
    
actions = all_actions(qc1)

#actions
objects = len(qc1.data) + 3 +1 # 3: registers, 1: grover generation
links = len(actions)
params = dict(qc1.count_ops())["cp"] + 3 + 3 + 1 
print("Objects:", objects, "Links:", links, "Params: ", params)


#Halstaed
un_opnd = 3+1+1 + qubits + count_params(int(qubits/2))[0]  # names of circuit/register, number of bits/qubits for registers
ov_opnd = 3 + 3+1
for i in range(int(qubits/2)):
    ov_opnd += actions.count((i, "q0"))
    ov_opnd += actions.count((i, "q1"))
ov_opnd+=count_params(int(qubits/2))[1]    
print("un_opnd:", un_opnd, "ov_opnd: ", ov_opnd)

un_optr = 3 + len(qc1.count_ops())
ov_optr = 4 + sum(qc1.count_ops().values())
print("un_optr:", un_optr, "ov_optr: ", ov_optr)

#MICOSE4aPS
sloc = qc1.qasm().count("\n")
ops = sum(qc1.count_ops().values())
num_qubits = len(actions)
params = count_params(int(qubits/2))[1]
names_numBits = 3+1 #numBits, numQubits, name of circuit
print("SLOC:", sloc, "Ops: ", ops, "Qubits: ", num_qubits, "Params: ", params, "Names_numBits: ", names_numBits)

In [ ]:
# QAOA - determine inputs

In [ ]:
# prepare circuit as it would be created using IBM Composer / QASM

qubits = 4

c_gate=Composite_Gates_final.CompositeGate()
l_gate=Loop_Operations_final.LoopOperation()
e_gate=Elementary_Gates.ElementaryGate()
m_gate=Measurements.MeasurementGate()

def w_create(qubits):
    w = []
    cols = qubits
    for i in range(qubits):
        row = list(np.random.random(cols)*10)
        w.append(row)
        cols-=1
    
    return w

w = w_create(qubits)   


#create overall circuit
qr = QuantumRegister(qubits, name="q0")
cr = ClassicalRegister(qubits)
qc1 = QuantumCircuit(qr, cr)
for i in range(qubits):
    qc1.h(i)

qc1.append(c_gate.cost_unitary(w), range(qubits))
qc1.append(c_gate.mixer_unitary(qubits), range(qubits))
qc1.append(c_gate.cost_unitary(w), range(qubits))
qc1.append(c_gate.mixer_unitary(qubits), range(qubits))

qc1.measure(range(qubits), range(qubits))

#create cost_unitary
cost_qc = QuantumCircuit(qubits)
cost_qc.append(c_gate.cost_unitary(w), range(qubits))
c_qc = cost_qc.decompose()

#create mixer_unitary
mixer_qc = QuantumCircuit(qubits)
mixer_qc.append(c_gate.mixer_unitary(qubits), range(qubits))
m_qc = mixer_qc.decompose()

In [ ]:
# calculate inputs for metrics (1x overall QC, 2x cost, 2x mixer)

def all_actions(qc):
    actions = []
    for gate in qc.data:
        indices = [(gate[1][i].index, gate[1][i].register.name) for i in range(len(gate[1]))]
        for index in indices:
            actions.append(index)
    return actions

    
ov_actions = all_actions(qc1)
c_actions = all_actions(c_qc)
m_actions = all_actions(m_qc)

#actions
objects = (2+len(qc1.data)) + 2*(len(c_qc.data)+1) + 2*(len(m_qc.data)+1)
links = (qubits+4*qubits+qubits) + 2*(2*c_qc.count_ops()["cx"]+c_qc.count_ops()["rz"]) + 2*(m_qc.count_ops()["rx"])
params = (2+2+1) + 2*(c_qc.count_ops()["rz"]+1) + 2*(m_qc.count_ops()["rx"]+1)
print("Objects:", objects, "Links:", links, "Params: ", params)

#Halstaed
un_opnd = 2+1+1 + qubits + 4 + 2*c_qc.count_ops()["rz"] + 2  # names of circuit/register, number of bits/qubits for registers, 4: names of composite circuits, 2: betas are all same in mixer
ov_opnd = 2+2+1 + (len(ov_actions) + 2*len(c_actions) + 2*len(m_actions)) + 4 + (2*c_qc.count_ops()["rz"] + 2*m_qc.count_ops()["rx"])
print("un_opnd:", un_opnd, "ov_opnd: ", ov_opnd)

un_optr = 3+1+ len(qc1.count_ops()) + len(c_qc.count_ops()) + len(m_qc.count_ops())   #1:"gate"
ov_optr = 4+4 + sum(qc1.count_ops().values()) + 2*sum(c_qc.count_ops().values()) + 2*sum(m_qc.count_ops().values())
print("un_optr:", un_optr, "ov_optr: ", ov_optr)

#MICOSE4aPS
sloc = qc1.qasm().count("\n")
ops = sum(qc1.count_ops().values()) + 2*sum(c_qc.count_ops().values()) + 2*sum(m_qc.count_ops().values())
num_qubits = (len(ov_actions) + 2*len(c_actions) + 2*len(m_actions))
params = (2*c_qc.count_ops()["rz"] + 2*m_qc.count_ops()["rx"])
names_numBits = 2+1 #numBits, numQubits, name of circuit
print("SLOC:", sloc, "Ops: ", ops, "Qubits: ", num_qubits, "Params: ", params, "Names_numBits: ", names_numBits)

In [ ]:
# compute Halstead and MICOSE4aPS

In [ ]:
# Halstead

def vocabulary(un_opn, un_opt):
    return un_opn + un_opt

def length(ov_opn, ov_opt):
    return ov_opn + ov_opt

def volume(ov_opn, ov_opt, un_opn, un_opt):
    l = length(ov_opn, ov_opt)
    voc = vocabulary(un_opn, un_opt)
    return l * np.log2(voc)
    
def difficulty(ov_opn, ov_opt, un_opn, un_opt):
    return (un_opt/2)*(ov_opn/un_opn)

def effort(ov_opn, ov_opt, un_opn, un_opt):
    return difficulty(ov_opn, ov_opt, un_opn, un_opt) * volume(ov_opn, ov_opt, un_opn, un_opt)

def time(ov_opn, ov_opt, un_opn, un_opt): # in seconds
    return effort(ov_opn, ov_opt, un_opn, un_opt)/18

In [ ]:
un_opn = 52 #number of unique operands
ov_opn = 1114542 #overall number of operands

un_opt = 8 #number of unique operators
ov_opt = 65731 #overall number of operators

print("Vocabulary: ", vocabulary(un_opn, un_opt), "Length: ", length(ov_opn, ov_opt), "Volume: ", volume(ov_opn, ov_opt, un_opn, un_opt), "Difficulty: ", difficulty(ov_opn, ov_opt, un_opn, un_opt), "Effort: ", effort(ov_opn, ov_opt, un_opn, un_opt), "Time: ", time(ov_opn, ov_opt, un_opn, un_opt))

In [ ]:
#### MICOSE4aPS
import numpy as np

def delta_pou(ke, kl, p,w,items_before, items_changed):
    return kl * w * (items_changed/max(items_changed, items_before)) + ke*w*(1 - np.exp(-p *(items_changed/max(items_changed, items_before))))

def maturity(deltas:list):
    return 1 - np.sum(np.array(deltas))/len(deltas)

#determine ke and kl
def determine_params(SLOC, change_type, s2): 
    if SLOC <= 150:
        ke = 0
    elif 150<SLOC<1000:
        ke = (1/850) * (SLOC-150)
    elif SLOC > 1000:
        ke = 1
    kl = 1 - ke


    # compute parameters
    p = 5
    if change_type=="quant":
        s1 = 1
    elif change_type=="non-quant":
        s1 = 0.5

    w = 0.8*s1 + 0.2*s2
    

    return ke, kl, w, p

In [ ]:
deltas = []

In [ ]:
# steps: 1. count SLOC for whole program, 2. determine change type for change
# 3. count items_before, items_changed for each change, 4. compute delta_pou for each change and store in list, 5. compute maturity

#inputs:
SLOC = 124 #SLOC after changes
change_type = "non-quant" # quant, non-quant
items_before = 5 # count the items of change i before (e.g., i...X-gate (operator change))
items_changed = 3

s2 = s2_func = s2_struc = s2_op = 1 # flexible parameters to further adjust weighting; default: 1 (defined by us)
ke, kl, w, p = determine_params(SLOC, change_type, s2)
print(ke, kl, w, p)
delta = delta_pou(ke, kl, p,w,items_before, items_changed)
print(delta)
deltas.append(delta)

In [ ]:
maturity = 1 - np.sum(np.array(deltas))/len(deltas)
print(deltas)
print(maturity)